# 用机器学习方法完成中文文本分类

* 输入形态：

CountVectorizer <br>
bow : bag of words <br>
ti-idf : text features <br>
word2vec : cbow/skip-gram (单个维度没有物理含义) <br>
* 传统机器学习模型：

Naive Bayes <br>
SVM <br>
RF <br>
GBDT
* 深度学习模型：

MLP <br>
CNN <br>
LSTM

## 朴素贝叶斯

我们试试用朴素贝叶斯完成一个中文文本分类器，一般在数据量足够，数据丰富度够的情况下，用朴素贝叶斯完成这个任务，准确度还是很不错的。

机器学习的算法要取得好效果，离不开数据，咱们先把数据加载进来看看。

### 准备数据

准备好数据，我们挑选 科技、汽车、娱乐、军事、运动 总共5类文本数据进行处理。

In [27]:
# 导入库
import jieba
import pandas as pd

#停用词
stopwords_data_path = "data/stopwords.txt"
stopwords_data_df = pd.read_csv(stopwords_data_path,encoding="utf-8",sep="\t",index_col=None,quoting=3,names=["stopword"])

# 准备数据
df_technology = pd.read_csv("./data/technology_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_technology = df_technology.dropna()

df_car = pd.read_csv("./data/car_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_car = df_car.dropna()

df_entertainment = pd.read_csv("./data/entertainment_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv("./data/military_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_military = df_military.dropna()

df_sports = pd.read_csv("./data/sports_news.csv", encoding='utf-8', names=["id", "content"], header=0)
df_sports = df_sports.dropna()

stopwords = stopwords_data_df.stopword.values.tolist()
technology = df_technology.content.apply(lambda line: line.strip()).values.tolist()
car = df_car.content.apply(lambda line: line.strip()).values.tolist()
entertainment = df_entertainment.content.apply(lambda line: line.strip()).values.tolist()
military = df_military.content.apply(lambda line: line.strip()).values.tolist()
sports = df_sports.content.apply(lambda line: line.strip()).values.tolist()

我们抽取几天数据来看下数据内容

In [28]:
technology = df_technology.content.apply(lambda line:line.strip()).values.tolist()
car = df_car.content.apply(lambda line:line.strip()).values.tolist()
entertainment = df_entertainment.content.apply(lambda line:line.strip()).values.tolist()
military = df_military.content.apply(lambda line:line.strip()).values.tolist()
sports = df_sports.content.apply(lambda line:line.strip()).values.tolist()

In [29]:
technology[20]

'2016年12月底，IDC发布调查报告显示，中国手机厂商于去年10月份在印度最主要的30个城市拿下了40%的市场份额。尽管三星仍然占据第一的位置，但二三名都被中国手机厂商拿下。其中，三星市场份额为26.1%，较上月增长15.8%，联想(包括摩托罗拉)以13.4%的份额位居第二，较上个月增长了50%，小米以10.7%的份额位列第三，较上月增长41.7%。'

In [30]:
car[10]

'从长远来看，当前我国新能源汽车市场还处于“政策驱动”的阶段，伴随着政府补贴政策进入退坡通道，行业或将进入发展的调整期，这将倒逼新能源汽车企业强化对核心技术的突破创新，推动车企通过提高自身综合实力来降本增效，大力提升新能源汽车的性价比，在退坡的周期内逐渐走上依托市场自行发展的道路。'

In [31]:
entertainment[4]

'网络综艺与电视综艺在播出模式、观众群体以及节目板块等方面也都存在差异。在传播上，电视台比网络平台更容易产生爆款。尽管这几年电视综艺也在网站上线，但由于观众观看的时间相对一致，而网络节目的时间较随意，所以同一档节目在同一时间，电视聚集观众的能力要比互联网强。《爸爸去哪儿4》转成网络综艺播出时，虽然也因一些话题而引发讨论，但是因传统受众观看习惯的影响，而网络观看又具有随意性，缺乏即时的、一致的观看体验，而导致没有掀起前几季的全民讨论的热潮。'

In [32]:
military[100]

'火箭军某导弹旅参谋长 陈伟：'

In [33]:
sports[900]

'“明年会继续争取吗？”媒体追问。此时的小德情绪显然有了波动，“到时候再说吧。我理解大家的工作，但也请你们理解我现在的处境。”'

### 分词和中文处理

5类数据最终处理的格式：<br>
word1 word2 word3 technology <br>
word1 word2 word3 word4 technology <br>
word1 word2 word4 car <br>

In [ ]:
'''
5类数据最终处理的格式：
word1 word2 word3 technology 
word1 word2 word3 word4 technology 
word1 word2 word4 car 
'''

'''
    lines: 文章的一条记录
    sentences: 返回的数据列表
    category： 文章的类别
'''
def preprocess_text(lines, sentences, category):
    for line in lines:
        segs = jieba.lcut(line)
        segs = [seg for seg in segs if len(seg) > 1 and seg not in stopwords]
        data = " ".join(segs)
        label = category
        sentences.append((data, label))


sentences = []
preprocess_text(technology, sentences, "technology")
preprocess_text(car, sentences, "car")
preprocess_text(entertainment, sentences, "entertainment")
preprocess_text(military, sentences, "military")
preprocess_text(sports, sentences, "sports")

## 样本顺序打乱 并打印训练格式的样本
import random
random.shuffle(sentences)
for data_label in sentences[0:100]:
    print(data_label[1] + " " + data_label[0])